# Discover how to use the Agents SDK framework from OpenAI for financial analysis

OpenAI recently released this new feature to help developers orchestrate multi-agent workflows, offering significant improvements over Swarm, the experimental agent SDK introduced last year.

This new SDK comes with some improvements that include:

- Agents: Easily configurable LLMs with clear instructions and built-in tools. The built-in tools include: web search, file search and computer use.
- Handoffs: Intelligently transfer control between agents. In Swarm, you had to explicitly define methods to transfer control between agents...
- Guardrails: Configurable safety checks for input and output validation.
- Tracing & Observability: Visualize agent execution traces to debug and optimize performance. You have access to all "Logs" and "Traces" of your agent in your OpenAI Dashboard, which gives a clear overview of which agent was called and the intermediate responses.


## Use case:
Retrieve financial information for a given stock or multiple stocks from multiple API endpoints, all in one place."


## Multi-agent framework:

See hereafter the details: ⏬






## Key Takeaways:  
- **Clear Design**: More straightforward to build compared to Swarm.  
- **Enhanced Capabilities:** Supports built-in functions for greater flexibility (Even though I didn't use them in this notebook, they can be highly useful for tasks like web searching or reading a PDF file.).  
- **Traces and Logs (Accessible via Dashboard)**: Provides a comprehensive record of events during agent execution, aiding in debugging, visualization, and workflow monitoring.

In [ ]:
!pip install openai-agents -q

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

import requests

FINANCIAL_MODELING_PREP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')

In [ ]:
from agents import Agent, Runner, function_tool
import nest_asyncio
nest_asyncio.apply()

# **Tools**

In [ ]:
@function_tool
def get_stock_price(symbol: str) -> dict():
    """
    Fetch the current stock price for the given symbol, the current volume, the average price 50d and 200d, EPS, PE and the next earnings Announcement.
    """
    url = f"https://financialmodelingprep.com/api/v3/quote-order/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        price = data[0]['price']
        volume = data[0]['volume']
        priceAvg50 = data[0]['priceAvg50']
        priceAvg200 = data[0]['priceAvg200']
        eps = data[0]['eps']
        pe = data[0]['pe']
        earningsAnnouncement = data[0]['earningsAnnouncement']
        return {"symbol": symbol.upper(), "price": price, "volume":volume,"priceAvg50":priceAvg50, "priceAvg200":priceAvg200, "EPS":eps, "PE":pe, "earningsAnnouncement":earningsAnnouncement }
    except (IndexError, KeyError):
        return {"error": f"Could not fetch price for symbol: {symbol}"}

@function_tool
def get_company_financials(symbol: str) -> dict():
    """
    Fetch basic financial information for the given company symbol such as the industry, the sector, the name of the company, and the market capitalization.
    """
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        results = data[0]
        financials = {
            "symbol": results["symbol"],
            "companyName": results["companyName"],
            "marketCap": results["mktCap"],
            "industry": results["industry"],
            "sector": results["sector"],
            "website": results["website"],
            "beta":results["beta"],
            "price":results["price"],
        }
        return {'financials': financials}
    except (IndexError, KeyError):
        return {"error": f"Could not fetch financials for symbol: {symbol}"}

@function_tool
def get_income_statement(symbol : str) -> dict():
    """
    Fetch last income statement for the given company symbol such as revenue, gross profit, net income, EBITDA, EPS.
    """
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=annual&apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        results = data[0]
        financials = {
            "date": results["date"],
            "revenue": results["revenue"],
            "gross profit": results["grossProfit"],
            "net Income": results["netIncome"],
            "ebitda": results["ebitda"],
            "EPS": results["eps"],
            "EPS diluted":results["epsdiluted"]
        }
        return {'financials': financials}
    except (IndexError, KeyError):
        return {"error": f"Could not fetch financials for symbol: {symbol}"}

# **Multi-Agent Framework**


**I created 4 agents:**

-  **stock_price_agent**: This agent is responsible of fetching the last available price, volume, average price 50d, 200d, EPS, PE, and the next earnings annoucement.

-  **company_basic_information_agent**: This agent is responsible of collecting the company basic data like description market capitalization, sector, industry...

-  **income_statement_agent**: This agent is responsible of gathering the last income statement data such as revenue, gross profit, net income, EBITDA, EPS.

-  **manager_agent**: This agent will handle determning and switching between agents to transfer the user's request to the best suited agent.


**Description and Tools:**

For each agent, I assigned a name, provided a description, specified a tool if needed, and defined handoff agents where necessary.  

I did not use built-in tools; instead, I implemented custom functions for financial analysis.

By default the model called is gpt-4o-2024-08-06

In [ ]:
# MODEL = "gpt-4o"
stock_price_agent = Agent(
    name="Stock Price Agent",
    instructions="Fetch Historical prices for a given stock symbol, the current volume, the average price 50d and 200d, EPS, PE and the next earnings Announcement.",
    # model = MODEL
    tools=[get_stock_price],
)

company_basic_information_agent = Agent(
    name="Company basic information Agent",
    instructions="Fetch basic financial information for the given company symbol such as the industry, the sector, the name of the company, and the market capitalization.",
    tools=[get_company_financials],
)

income_statement_agent = Agent(
    name="Income Statement Agent",
    instructions="Fetch last income statement for the given company symbol such as revenue, gross profit, net income, EBITDA, EPS.",
    tools=[get_income_statement],
)

manager_agent = Agent(
    name="Manager Agent",
    instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
    handoffs=[stock_price_agent, company_basic_information_agent, income_statement_agent]
)

# First Call

In [ ]:
output = Runner.run_sync(
    starting_agent=manager_agent,
    input="What is the current stock price of Amazon and Nvidia?",
)

In [ ]:
print(output.final_output)

Here are the current stock details:

**Amazon (AMZN):**
- Current Price: $197.95
- Volume: 36,633,353
- Average Price (50 days): $221.40
- Average Price (200 days): $199.38
- EPS: $5.52
- PE Ratio: 35.86
- Next Earnings Announcement: April 28, 2025

**Nvidia (NVDA):**
- Current Price: $121.67
- Volume: 274,061,576
- Average Price (50 days): $129.30
- Average Price (200 days): $127.64
- EPS: $2.94
- PE Ratio: 41.38
- Next Earnings Announcement: May 28, 2025


In [ ]:
from agents import items

## **Debugging intermediate steps**

By accessing the detailed output from OpenAI, you can view the intermediate steps, including which agent was selected by the `manager_agent`, which tool was used, the intermediate outputs, and the final result.  

This information is also available in a well-organized format directly in your OpenAI **dashboard** under **Logs** and **Traces**.

In [ ]:
for item in output.new_items[:]:
  # print(item)
  if type(item) == items.HandoffCallItem:
    print("HandoffCallItem")
    print(item.agent.name)
    print(item.agent.instructions)
    print(item.raw_item.name)

  if type(item) == items.MessageOutputItem:
    print("MessageOutputItem")
    print(item.raw_item.content[0].text)

  if type(item) == items.ToolCallItem:
    print("ToolCallItem")
    print(item.agent.name)


  if type(item) == items.ToolCallOutputItem or type(item) == items.HandoffOutputItem:
    print("ToolCallOutputItem or HandoffOutputItem")
    print(item.raw_item['output'])
  print("\n\n")


HandoffCallItem
Manager Agent
Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.
transfer_to_stock_price_agent



HandoffCallItem
Manager Agent
Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.
transfer_to_stock_price_agent



ToolCallOutputItem or HandoffOutputItem
Multiple handoffs detected, ignoring this one.



ToolCallOutputItem or HandoffOutputItem
{'assistant': 'Stock Price Agent'}



ToolCallItem
Stock Price Agent



ToolCallItem
Stock Price Agent



ToolCallOutputItem or HandoffOutputItem
{'symbol': 'AMZN', 'price': 197.95, 'volume': 36633353, 'priceAvg50': 221.3966, 'priceAvg200': 199.38335, 'EPS': 5.52, 'PE': 35.86, 'earningsAnnouncement': '2025-04-28T10:59:00.000+0000'}



ToolCallOutputItem or HandoffOutputItem
{'symbol': 'NVDA', 'price': 121.67, 'volume': 274061576, 'priceAvg50': 129.2964, 'priceAvg200': 127.63886, 'EPS': 2.94, 'PE': 41.38, 'earningsA

# **Second Call**

It does not handle memory for now, unlike Assistants AI where you can use Thread to memorize chat history... OpenAI is planning to develop all Assistant AI features in Responses API (which can be used in Agents SDK), so they can deprecate Assistant AI by mid-2026.


In [ ]:
%%time
output = Runner.run_sync(
    starting_agent=manager_agent,
    input="What is their market capitalization?",
)
print(output.final_output)

Could you please provide the company's name or symbol so I can retrieve the market capitalization for you?
CPU times: user 49.7 ms, sys: 1.87 ms, total: 51.5 ms
Wall time: 1.73 s


# **Third Call**

In [ ]:
%%time
output = Runner.run_sync(
    starting_agent=manager_agent,
    input="What is the market capitalization of Amazon and Nvidia?",
)
print(output.final_output)

Here is the market capitalization for the companies you requested:

1. **Amazon.com, Inc. (AMZN)**
   - Market Capitalization: $2.10 trillion
   - Industry: Specialty Retail
   - Sector: Consumer Cyclical
   - [Website](https://www.amazon.com)

2. **NVIDIA Corporation (NVDA)**
   - Market Capitalization: $2.97 trillion
   - Industry: Semiconductors
   - Sector: Technology
   - [Website](https://www.nvidia.com)
CPU times: user 133 ms, sys: 12.8 ms, total: 145 ms
Wall time: 6.28 s


## **Debugging intermediate steps**

In [ ]:
for item in output.new_items[:]:
  # print(item)
  if type(item) == items.HandoffCallItem:
    print("HandoffCallItem")
    print(item.agent.name)
    print(item.agent.instructions)
    print(item.raw_item.name)

  if type(item) == items.MessageOutputItem:
    print("MessageOutputItem")
    print(item.raw_item.content[0].text)

  if type(item) == items.ToolCallItem:
    print("ToolCallItem")
    print(item.agent.name)


  if type(item) == items.ToolCallOutputItem or type(item) == items.HandoffOutputItem:
    print("ToolCallOutputItem or HandoffOutputItem")
    print(item.raw_item['output'])
  print("\n\n")


HandoffCallItem
Manager Agent
Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.
transfer_to_company_basic_information_agent



HandoffCallItem
Manager Agent
Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.
transfer_to_company_basic_information_agent



ToolCallOutputItem or HandoffOutputItem
Multiple handoffs detected, ignoring this one.



ToolCallOutputItem or HandoffOutputItem
{'assistant': 'Company basic information Agent'}



ToolCallItem
Company basic information Agent



ToolCallItem
Company basic information Agent



ToolCallOutputItem or HandoffOutputItem
{'symbol': 'AMZN', 'companyName': 'Amazon.com, Inc.', 'marketCap': 2097814715000, 'industry': 'Specialty Retail', 'sector': 'Consumer Cyclical', 'website': 'https://www.amazon.com', 'beta': 1.192, 'price': 197.95}



ToolCallOutputItem or HandoffOutputItem
{'symbol': 'NVDA', 'companyName': 'NVIDIA Corpo

# **Fourth Call**

In [ ]:
%%time
output = Runner.run_sync(
    starting_agent=manager_agent,
    input="What is the last revenue of Nvidia?",
)
print(output.final_output)

Nvidia's last reported revenue is $130.497 billion.
CPU times: user 84.9 ms, sys: 6.15 ms, total: 91.1 ms
Wall time: 2.98 s


## **Debugging intermediate steps**

In [ ]:
for item in output.new_items[:]:
  # print(item)
  if type(item) == items.HandoffCallItem:
    print("HandoffCallItem")
    print(item.agent.name)
    print(item.agent.instructions)
    print(item.raw_item.name)

  if type(item) == items.MessageOutputItem:
    print("MessageOutputItem")
    print(item.raw_item.content[0].text)

  if type(item) == items.ToolCallItem:
    print("ToolCallItem")
    print(item.agent.name)


  if type(item) == items.ToolCallOutputItem or type(item) == items.HandoffOutputItem:
    print("ToolCallOutputItem or HandoffOutputItem")
    print(item.raw_item['output'])
  print("\n\n")


HandoffCallItem
Manager Agent
Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.
transfer_to_income_statement_agent



ToolCallOutputItem or HandoffOutputItem
{'assistant': 'Income Statement Agent'}



ToolCallItem
Income Statement Agent



ToolCallOutputItem or HandoffOutputItem
{'financials': {'date': '2025-01-26', 'revenue': 130497000000, 'gross profit': 97858000000, 'net Income': 72880000000, 'ebitda': 86137000000, 'EPS': 2.97, 'EPS diluted': 2.94}}



MessageOutputItem
Nvidia's last reported revenue is $130.497 billion.



